# Import Package 

In [1]:
import re
import json
import math
import numpy as np
import collections
import xml.dom.minidom
from stemming.porter2 import stem

#  Input from .txt File into dictionary

In [2]:
# Term -> (Document -> Position)
# input term_doc_pos_dict
# {'Term1': {doc1: [position1, position2], doc2: [position1, position2]}
#  'Term2': {doc1: [position1, position2, position3], doc2: [position1, position2]}}

file = open('index_ts.txt', 'r') 
js = file.read()
term_doc_pos_dict = json.loads(js) 
file.close() 

# term_doc_pos_dict

In [3]:
# Term -> Position
# input from term_position_dict
# {'Term1': [[doc1, pos1], [doc2, pos2], [doc3, pos3]
#  'Term2': [[doc1, pos1], [doc2, pos2], [doc3, pos3]}

file = open('index_ts_pos.txt', 'r') 
js = file.read()
term_position_dict = json.loads(js) 
file.close() 

# term_position_dict

# Get Score of each Query

In [4]:
# stopping words list
with open('englishST.txt', 'r') as eng:
    eng_str = eng.read()
eng_list = eng_str.split()  

def takeSecond(elem):
    return elem[1]

def getScore(s, term_doc_pos_dict):
    s_list = s.split()
    for m in range(len(s_list)):
        s_list[m] = s_list[m].lower()  # lower()

    # Stopping 
    stop_list = []
    for i in s_list:
        if (i not in eng_list):
            stop_list.extend([i])
    
    # Normalisation
    w_list = []
    for j in stop_list:
        w_list.extend([stem(j)])
    
    print(w_list)
    
    # get document list
    doc_list = list(term_doc_pos_dict[w_list[0]].keys()) 
    for i in range(len(w_list)-1):
        doc_list = list(set(doc_list).union(term_doc_pos_dict[w_list[i+1]].keys())) 
    for j in range(len(doc_list)):
        doc_list[j] = int(doc_list[j])
    doc_list.sort()
    
    # get score list
    score_list = []
    for i in range(len(doc_list)):
        score_doc = 0
        for j in range(len(w_list)):           
            if (str(doc_list[i]) in list(term_doc_pos_dict[w_list[j]].keys())):
                a = 1 + math.log10(len(term_doc_pos_dict[w_list[j]][str(doc_list[i])]))
                # print(a)
                b = math.log10(5000/len(term_doc_pos_dict[w_list[j]].keys()))
                # print(b)
                score_doc = score_doc + a * b
        score_list.extend([[int(doc_list[i]), format(score_doc, '.4f')]])  
    
    # sort according to 1st column
    score_list.sort(key=takeSecond, reverse=True)
              
    return score_list

# Generate term_score_dict 

In [12]:
# Input query txt
with open('queries.lab3.txt', 'r') as f:
    line_list = f.readlines()
f.close()

term_score_dict = {}
num_list = []
l_list = []
r = """[0-9?]"""

# get the number
for i in range(len(line_list)):  
    num_list.extend([line_list[i].split()])  # split 
# get the line
for j in range(len(line_list)):
    l_list.extend([re.sub(r, '', line_list[j]).strip()])  # detele 0-9

for k in range(len(l_list)):
    term_score_dict[int(num_list[k][0])] = getScore(l_list[k], term_doc_pos_dict)

term_score_dict

['incom', 'tax', 'reduct']
['peac', 'middl', 'east']
['unemploy', 'rate', 'uk']
['industri', 'scotland']
['industri', 'comput']
['microsoft', 'window']
['stock', 'market', 'japan']
['educ', 'comput']
['health', 'industri']
['campaign', 'polit', 'parti']


{1: [[65, '8.4096'],
  [3533, '8.2741'],
  [3562, '6.0631'],
  [3608, '6.0087'],
  [92, '5.8455'],
  [141, '5.6335'],
  [361, '5.6335'],
  [3734, '5.5661'],
  [3829, '5.4892'],
  [3420, '5.4494'],
  [3405, '5.2652'],
  [3529, '5.1965'],
  [3387, '5.0595'],
  [3599, '5.0595'],
  [3706, '4.9739'],
  [3403, '4.5886'],
  [282, '4.5801'],
  [3596, '4.4959'],
  [3503, '4.4442'],
  [3818, '4.3593'],
  [163, '4.3534'],
  [177, '4.3373'],
  [3699, '4.2635'],
  [125, '4.2064'],
  [3417, '4.2064'],
  [3708, '4.1920'],
  [3495, '4.1021'],
  [3710, '4.1021'],
  [184, '4.0146'],
  [3838, '4.0146'],
  [3519, '3.9764'],
  [3817, '3.9764'],
  [314, '3.8339'],
  [3535, '3.8339'],
  [270, '3.7633'],
  [3407, '3.7633'],
  [3680, '3.7633'],
  [3890, '3.7633'],
  [3606, '3.6725'],
  [3345, '3.6324'],
  [3327, '3.5826'],
  [3441, '3.5826'],
  [3590, '3.5826'],
  [3705, '3.5826'],
  [3834, '3.5826'],
  [3329, '3.3705'],
  [33, '3.3337'],
  [170, '3.3337'],
  [268, '3.3337'],
  [3337, '3.3337'],
  [3622, '3.33

# Output rank txt

In [14]:
# Output to output_rank.txt
# clear
file = open('results.ranked.txt', 'w').close()

# write
doc = open('results.ranked.txt', 'w')
for key, value in term_score_dict.items():
    if (len(value) > 150):
        for i in range(150):
            print(str(key) + "," + str(value[i][0]) + "," + str(value[i][1]), file=doc)
    else:
        for i in range(len(value)):
            print(str(key) + "," + str(value[i][0]) + "," + str(value[i][1]), file=doc)
doc.close()

In [13]:
# clear
file = open('ranked_dict.txt', 'w').close()
# wirte
js = json.dumps(term_score_dict) 
file = open('ranked_dict.txt', 'w') 
file.write(js) 
file.close()